In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import joblib
from sklearn.model_selection import ParameterGrid
from ultis import *

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

读取处理好的特征数据

In [4]:
df = pd.read_csv('data/training_past.txt', delimiter=';', parse_dates=['time_interval_begin'], dtype={'link_ID': object})
df.head(10)

,link_ID,date,time_interval_begin,travel_time,imputation1,lagging1,lagging2,lagging3,lagging4,lagging5,length,area,vacation,minute_series,day_of_week,day_of_week_en,hour_en,"week_hour_1.0,1.0","week_hour_1.0,2.0","week_hour_1.0,3.0","week_hour_2.0,1.0","week_hour_2.0,2.0","week_hour_2.0,3.0","week_hour_3.0,1.0","week_hour_3.0,2.0","week_hour_3.0,3.0",links_num_2,links_num_3,links_num_4,links_num_5,width_3,width_6,width_9,width_12,width_15,link_ID_en
0,3377906280028510514,2017-03-01,2017-03-01 06:00:00,1.662360,True,NaN,NaN,NaN,NaN,NaN,48,144,0.0,0.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
1,3377906280028510514,2017-03-01,2017-03-01 06:02:00,1.681661,True,1.662360,NaN,NaN,NaN,NaN,48,144,0.0,2.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
2,3377906280028510514,2017-03-01,2017-03-01 06:04:00,1.676155,True,1.681661,1.662360,NaN,NaN,NaN,48,144,0.0,4.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
3,3377906280028510514,2017-03-01,2017-03-01 06:06:00,1.683786,True,1.676155,1.681661,1.662360,NaN,NaN,48,144,0.0,6.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
4,3377906280028510514,2017-03-01,2017-03-01 06:08:00,1.683193,True,1.683786,1.676155,1.681661,1.662360,NaN,48,144,0.0,8.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
5,3377906280028510514,2017-03-01,2017-03-01 06:10:00,1.629241,False,1.683193,1.683786,1.676155,1.681661,1.662360,48,144,0.0,10.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
6,3377906280028510514,2017-03-01,2017-03-01 06:12:00,1.629241,False,1.629241,1.683193,1.683786,1.676155,1.681661,48,144,0.0,12.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
7,3377906280028510514,2017-03-01,2017-03-01 06:14:00,1.629241,False,1.629241,1.629241,1.683193,1.683786,1.676155,48,144,0.0,14.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
8,3377906280028510514,2017-03-01,2017-03-01 06:16:00,1.701797,True,1.629241,1.629241,1.629241,1.683193,1.683786,48,144,0.0,16.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47
9,3377906280028510514,2017-03-01,2017-03-01 06:18:00,1.696063,True,1.701797,1.629241,1.629241,1.629241,1.683193,48,144,0.0,18.0,3,1.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,47


时间序列特征

In [4]:
lagging = 5
lagging_feature = ['lagging%01d' % e for e in range(lagging, 0, -1)]
lagging_feature

['lagging5', 'lagging4', 'lagging3', 'lagging2', 'lagging1']

In [8]:
base_feature = [x for x in df.columns.values.tolist() if x not in ['time_interval_begin', 'link_ID', 'link_ID_int',
                                                                   'date', 'travel_time', 'imputation1',
                                                                   'minute_series', 'area', 'hour_en', 'day_of_week']]

In [9]:
base_feature = [x for x in base_feature if x not in lagging_feature]

In [10]:
train_feature = list(base_feature)
train_feature.extend(lagging_feature)
valid_feature = list(base_feature)
valid_feature.extend(['minute_series', 'travel_time'])
print (train_feature)

['length', 'vacation', 'day_of_week_en', 'week_hour_1.0,1.0', 'week_hour_1.0,2.0', 'week_hour_1.0,3.0', 'week_hour_2.0,1.0', 'week_hour_2.0,2.0', 'week_hour_2.0,3.0', 'week_hour_3.0,1.0', 'week_hour_3.0,2.0', 'week_hour_3.0,3.0', 'links_num_2', 'links_num_3', 'links_num_4', 'links_num_5', 'width_3', 'width_6', 'width_9', 'width_12', 'width_15', 'link_ID_en', 'lagging5', 'lagging4', 'lagging3', 'lagging2', 'lagging1']


xgboost训练参数：


In [8]:
params_grid = {
    'learning_rate': [0.05],
    'n_estimators': [100],
    'subsample': [0.6],
    'colsample_bytree': [0.6],
    'max_depth': [7],
    'min_child_weight': [1],
    'reg_alpha': [2],
    'gamma': [0]
}

In [9]:
grid = ParameterGrid(params_grid)

训练模块

In [10]:
def fit_evaluate(df, df_test, params):
    df = df.dropna()
    X = df[train_feature].values
    y = df['travel_time'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

    df_test = df_test[valid_feature].values
    valid_data = bucket_data(df_test)

    eval_set = [(X_test, y_test)]
    regressor = xgb.XGBRegressor(learning_rate=params['learning_rate'], n_estimators=params['n_estimators'],
                                 booster='gbtree', objective='reg:linear', n_jobs=-1, subsample=params['subsample'],
                                 colsample_bytree=params['colsample_bytree'], random_state=0,
                                 max_depth=params['max_depth'], gamma=params['gamma'],
                                 min_child_weight=params['min_child_weight'], reg_alpha=params['reg_alpha'])
    regressor.fit(X_train, y_train, verbose=False, early_stopping_rounds=10, eval_metric=mape_ln,
                  eval_set=eval_set)
    return regressor, cross_valid(regressor, valid_data,lagging=lagging), regressor.best_iteration, regressor.best_score

In [12]:
def train(df, params, best, vis=False):
    train1 = df.loc[df['time_interval_begin'] <= pd.to_datetime('2017-03-24')]
    train2 = df.loc[
        (df['time_interval_begin'] > pd.to_datetime('2017-03-24')) & (
            df['time_interval_begin'] <= pd.to_datetime('2017-04-18'))]
    train3 = df.loc[
        (df['time_interval_begin'] > pd.to_datetime('2017-04-18')) & (
            df['time_interval_begin'] <= pd.to_datetime('2017-05-12'))]
    train4 = df.loc[
        (df['time_interval_begin'] > pd.to_datetime('2017-05-12')) & (
            df['time_interval_begin'] <= pd.to_datetime('2017-06-06'))]
    train5 = df.loc[
        (df['time_interval_begin'] > pd.to_datetime('2017-06-06')) & (
            df['time_interval_begin'] <= pd.to_datetime('2017-06-30'))]

    regressor, loss1, best_iteration1, best_score1 = fit_evaluate(pd.concat([train1, train2, train3, train4]), train5,
                                                                  params)
    print (best_iteration1, best_score1, loss1)

    regressor, loss2, best_iteration2, best_score2 = fit_evaluate(pd.concat([train1, train2, train3, train5]), train4,
                                                                  params)
    print (best_iteration2, best_score2, loss2)

    regressor, loss3, best_iteration3, best_score3 = fit_evaluate(pd.concat([train1, train2, train4, train5]), train3,
                                                                  params)
    print (best_iteration3, best_score3, loss3)

    regressor, loss4, best_iteration4, best_score4 = fit_evaluate(pd.concat([train1, train3, train4, train5]), train2,
                                                                  params)
    print (best_iteration4, best_score4, loss4)

    regressor, loss5, best_iteration5, best_score5 = fit_evaluate(pd.concat([train2, train3, train4, train5]), train1,
                                                                  params)
    print (best_iteration5, best_score5, loss5)
    
    loss = [loss1, loss2, loss3, loss4, loss5]
    params['loss_std'] = np.std(loss)
    params['loss'] = str(loss)
    params['mean_loss'] = np.mean(loss)
    params['n_estimators'] = str([best_iteration1, best_iteration2, best_iteration3, best_iteration4, best_iteration5])
    params['best_score'] = str([best_score1, best_score2, best_score3, best_score4, best_score5])
    
    print (str(params))
    if np.mean(loss) <= best:
        best = np.mean(loss)
        print ("best with: " + str(params))
        #feature_vis(regressor, train_feature)
    return best

In [13]:
best = 1
for params in grid:
    best = train(df, params, best)

D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\core.py:160: UserWarning: [16:43:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


99 0.179128 0.07066397328575548


D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\core.py:160: UserWarning: [16:44:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


99 0.15276 0.23204371552714265


D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\core.py:160: UserWarning: [16:44:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


99 0.14545 0.28501998531521683


D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\core.py:160: UserWarning: [16:45:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


99 0.144213 0.2956625393992471


D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\core.py:160: UserWarning: [16:46:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


99 0.145541 0.2995498470886828
{'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': '[99, 99, 99, 99, 99]', 'reg_alpha': 2, 'subsample': 0.6, 'loss_std': np.float64(0.08643041797661757), 'loss': '[np.float64(0.07066397328575548), np.float64(0.23204371552714265), np.float64(0.28501998531521683), np.float64(0.2956625393992471), np.float64(0.2995498470886828)]', 'mean_loss': np.float64(0.23658801212320898), 'best_score': '[0.179128, 0.15276, 0.14545, 0.144213, 0.145541]'}
best with: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': '[99, 99, 99, 99, 99]', 'reg_alpha': 2, 'subsample': 0.6, 'loss_std': np.float64(0.08643041797661757), 'loss': '[np.float64(0.07066397328575548), np.float64(0.23204371552714265), np.float64(0.28501998531521683), np.float64(0.2956625393992471), np.float64(0.2995498470886828)]', 'mean_loss': np.float64(0.23658801212320898), 'best_score'

### 生成预测序列

In [13]:
submit_params = {
     'learning_rate': 0.05,
     'n_estimators': 100,
     'subsample': 0.6,
     'colsample_bytree': 0.6,
     'max_depth': 7,
     'min_child_weight': 1,
     'reg_alpha': 2,
     'gamma': 0
}

In [14]:
def xgboost_submit(df, params):
    train_df = df.loc[df['time_interval_begin'] < pd.to_datetime('2017-07-01')]

    train_df = train_df.dropna()
    X = train_df[train_feature].values
    y = train_df['travel_time'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

    eval_set = [(X_test, y_test)]
    regressor = xgb.XGBRegressor(learning_rate=params['learning_rate'], n_estimators=params['n_estimators'],
                                 booster='gbtree', objective='reg:linear', n_jobs=-1, subsample=params['subsample'],
                                 colsample_bytree=params['colsample_bytree'], random_state=0,
                                 max_depth=params['max_depth'], gamma=params['gamma'],
                                 min_child_weight=params['min_child_weight'], reg_alpha=params['reg_alpha'])
    regressor.fit(X_train, y_train, verbose=True, early_stopping_rounds=10, eval_metric=mape_ln,
                  eval_set=eval_set)
    joblib.dump(regressor, 'model/xgbr.pkl')
    print (regressor)
    submission(train_feature, regressor, df, 'submission/xgbr1.txt', 'submission/xgbr2.txt', 'submission/xgbr3.txt',
               'submission/xgbr4.txt')

In [17]:
xgboost_submit(df, submit_params)

D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
D:\Anaconda\envs\bigdata\lib\site-packages\xgboost\core.py:160: UserWarning: [17:43:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:0.88714	validation_0-mape:1.20941
[1]	validation_0-rmse:0.84555	validation_0-mape:1.12636
[2]	validation_0-rmse:0.80748	validation_0-mape:1.05303
[3]	validation_0-rmse:0.77178	validation_0-mape:0.98707
[4]	validation_0-rmse:0.73637	validation_0-mape:0.92491
[5]	validation_0-rmse:0.70416	validation_0-mape:0.86980
[6]	validation_0-rmse:0.67413	validation_0-mape:0.81998
[7]	validation_0-rmse:0.64602	validation_0-mape:0.77446
[8]	validation_0-rmse:0.61928	validation_0-mape:0.73256
[9]	validation_0-rmse:0.59230	validation_0-mape:0.69202
[10]	validation_0-rmse:0.56681	validation_0-mape:0.65460
[11]	validation_0-rmse:0.54410	validation_0-mape:0.62147
[12]	validation_0-rmse:0.52140	validation_0-mape:0.58943
[13]	validation_0-rmse:0.50002	validation_0-mape:0.55983
[14]	validation_0-rmse:0.47987	validation_0-mape:0.53234
[15]	validation_0-rmse:0.46093	validation_0-mape:0.50681
[16]	validation_0-rmse:0.44519	validation_0-mape:0.48532
[17]	validation_0-rmse:0.42978	validation

In [122]:
#将给定类似的原始数据转化成所需数据
import pandas as pd

df = pd.read_csv('data/step_data.txt', delimiter=';', dtype={'link_ID': object})
df['time_interval_begin'] = pd.to_datetime(df['time_interval'].map(lambda x: x[1:20]))

df = df.drop(['time_interval'], axis=1)
df

,link_ID,date,travel_time,time_interval_begin
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:00:00
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:02:00
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:04:00
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:06:00
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:08:00
5,3377906280028510514,2017-03-01,1.629241,2017-03-01 06:10:00
6,3377906280028510514,2017-03-01,1.629241,2017-03-01 06:12:00
7,3377906280028510514,2017-03-01,1.629241,2017-03-01 06:14:00
8,3377906280028510514,2017-03-01,1.701797,2017-03-01 06:16:00


In [123]:
df.iloc[[5]]

,link_ID,date,travel_time,time_interval_begin
5,3377906280028510514,2017-03-01,1.629241,2017-03-01 06:10:00


In [124]:
df1 = df.copy()

df1['time_interval_begin'] = df1['time_interval_begin'] + pd.DateOffset(minutes=5 * 2)

df1 = df1.rename(columns={'travel_time': 'lagging' + str(5)})


df1.head()

,link_ID,date,lagging5,time_interval_begin
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:10:00
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:12:00
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:14:00
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:16:00
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:18:00


In [125]:
df2 = pd.merge(df, df1[['link_ID', 'time_interval_begin', 'lagging' + str(5)]], on=['link_ID', 'time_interval_begin'],how='left')

df2.head()

,link_ID,date,travel_time,time_interval_begin,lagging5
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:00:00,NaN
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:02:00,NaN
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:04:00,NaN
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:06:00,NaN
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:08:00,NaN


In [126]:
def create_lagging(df, df_original, i):
    df1 = df_original.copy()
    df1['time_interval_begin'] = df1['time_interval_begin'] + pd.DateOffset(minutes=i * 2)
    df1 = df1.rename(columns={'travel_time': 'lagging' + str(i)})
    df2 = pd.merge(df, df1[['link_ID', 'time_interval_begin', 'lagging' + str(i)]],
                   on=['link_ID', 'time_interval_begin'],
                   how='left')
    return df2

In [127]:
df1 = create_lagging(df, df, 1)

In [128]:
lagging = 5
for i in range(2, lagging + 1):
    df1 = create_lagging(df1, df, i)

In [129]:
df1.head()

,link_ID,date,travel_time,time_interval_begin,lagging1,lagging2,lagging3,lagging4,lagging5
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:00:00,NaN,NaN,NaN,NaN,NaN
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:02:00,1.662360,NaN,NaN,NaN,NaN
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:04:00,1.681661,1.662360,NaN,NaN,NaN
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:06:00,1.676155,1.681661,1.662360,NaN,NaN
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:08:00,1.683786,1.676155,1.681661,1.66236,NaN


In [130]:
link_infos = pd.read_csv('gy_contest_link_info.txt', delimiter=';', dtype={'link_ID': object})
link_tops = pd.read_csv('gy_contest_link_top.txt', delimiter=',', dtype={'link_ID': object})

In [131]:
link_tops = link_tops.fillna(0)
link_infos = pd.merge(link_infos, link_tops, on=['link_ID'], how='left')
link_infos['links_num'] = link_infos["in_links"]+link_infos["out_links"]

In [132]:
link_infos['area'] = link_infos['length'] * link_infos['width']
df2 = pd.merge(df1, link_infos[['link_ID', 'length', 'width', 'links_num', 'area']], on=['link_ID'], how='left')

In [133]:
df2.head()

,link_ID,date,travel_time,time_interval_begin,lagging1,lagging2,lagging3,lagging4,lagging5,length,width,links_num,area
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:00:00,NaN,NaN,NaN,NaN,NaN,48,3,2,144
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:02:00,1.662360,NaN,NaN,NaN,NaN,48,3,2,144
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:04:00,1.681661,1.662360,NaN,NaN,NaN,48,3,2,144
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:06:00,1.676155,1.681661,1.662360,NaN,NaN,48,3,2,144
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:08:00,1.683786,1.676155,1.681661,1.66236,NaN,48,3,2,144


In [134]:
# 假期特征
df2.loc[df2['date'].isin(
    ['2017-04-02', '2017-04-03', '2017-04-04', '2017-04-29', '2017-04-30', '2017-05-01',
     '2017-05-28', '2017-05-29', '2017-05-30']), 'vacation'] = 1
df2.loc[~df2['date'].isin(
    ['2017-04-02', '2017-04-03', '2017-04-04', '2017-04-29', '2017-04-30', '2017-05-01',
     '2017-05-28', '2017-05-29', '2017-05-30']), 'vacation'] = 0

In [135]:
df2.head()

,link_ID,date,travel_time,time_interval_begin,lagging1,lagging2,lagging3,lagging4,lagging5,length,width,links_num,area,vacation
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:00:00,NaN,NaN,NaN,NaN,NaN,48,3,2,144,0.0
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:02:00,1.662360,NaN,NaN,NaN,NaN,48,3,2,144,0.0
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:04:00,1.681661,1.662360,NaN,NaN,NaN,48,3,2,144,0.0
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:06:00,1.676155,1.681661,1.662360,NaN,NaN,48,3,2,144,0.0
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:08:00,1.683786,1.676155,1.681661,1.66236,NaN,48,3,2,144,0.0


In [136]:
# 起始分钟特征
df2.loc[df2['time_interval_begin'].dt.hour.isin([6, 7, 8]), 'minute_series'] = \
    df2['time_interval_begin'].dt.minute + (df2['time_interval_begin'].dt.hour - 6) * 60

df2.loc[df2['time_interval_begin'].dt.hour.isin([13, 14, 15]), 'minute_series'] = \
    df2['time_interval_begin'].dt.minute + (df2['time_interval_begin'].dt.hour - 13) * 60

df2.loc[df2['time_interval_begin'].dt.hour.isin([16, 17, 18]), 'minute_series'] = \
    df2['time_interval_begin'].dt.minute + (df2['time_interval_begin'].dt.hour - 16) * 60

In [137]:
df2.head()

,link_ID,date,travel_time,time_interval_begin,lagging1,lagging2,lagging3,lagging4,lagging5,length,width,links_num,area,vacation,minute_series
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:00:00,NaN,NaN,NaN,NaN,NaN,48,3,2,144,0.0,0.0
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:02:00,1.662360,NaN,NaN,NaN,NaN,48,3,2,144,0.0,2.0
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:04:00,1.681661,1.662360,NaN,NaN,NaN,48,3,2,144,0.0,4.0
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:06:00,1.676155,1.681661,1.662360,NaN,NaN,48,3,2,144,0.0,6.0
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:08:00,1.683786,1.676155,1.681661,1.66236,NaN,48,3,2,144,0.0,8.0


In [138]:
# 星期特征
df2['day_of_week'] = df2['time_interval_begin'].map(lambda x: x.weekday() + 1)
df2.loc[df2['day_of_week'].isin([1, 2, 3]), 'day_of_week_en'] = 1
df2.loc[df2['day_of_week'].isin([4, 5]), 'day_of_week_en'] = 2
df2.loc[df2['day_of_week'].isin([6, 7]), 'day_of_week_en'] = 3

In [139]:
# 时间段特征
#这里有我的改动，因为原本只需要一部分的时间，但是为了可以随意选择时间，这里将全天的时间划分成三个部分
df2.loc[df['time_interval_begin'].dt.hour.isin([1, 2, 3, 4, 6, 7, 8]), 'hour_en'] = 1
df2.loc[df['time_interval_begin'].dt.hour.isin([9, 10, 11, 12, 13, 14, 15]), 'hour_en'] = 2
df2.loc[df['time_interval_begin'].dt.hour.isin([16, 17, 18, 19, 20, 21, 22, 23, 24]), 'hour_en'] = 3

In [140]:
# 星期，时间段合并特征
df2['week_hour'] = df2["day_of_week_en"].astype('str') + "," + df2["hour_en"].astype('str')

In [141]:
df2.head()

,link_ID,date,travel_time,time_interval_begin,lagging1,lagging2,lagging3,lagging4,lagging5,length,width,links_num,area,vacation,minute_series,day_of_week,day_of_week_en,hour_en,week_hour
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:00:00,NaN,NaN,NaN,NaN,NaN,48,3,2,144,0.0,0.0,3,1.0,1.0,"1.0,1.0"
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:02:00,1.662360,NaN,NaN,NaN,NaN,48,3,2,144,0.0,2.0,3,1.0,1.0,"1.0,1.0"
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:04:00,1.681661,1.662360,NaN,NaN,NaN,48,3,2,144,0.0,4.0,3,1.0,1.0,"1.0,1.0"
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:06:00,1.676155,1.681661,1.662360,NaN,NaN,48,3,2,144,0.0,6.0,3,1.0,1.0,"1.0,1.0"
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:08:00,1.683786,1.676155,1.681661,1.66236,NaN,48,3,2,144,0.0,8.0,3,1.0,1.0,"1.0,1.0"


In [142]:
df2 = pd.get_dummies(df2, columns=['week_hour', 'links_num', 'width'])

In [143]:
def mean_time(group):
    group['link_ID_en'] = group['travel_time'].mean()
    return group

In [144]:
df2 = df2.groupby('link_ID').apply(mean_time)

C:\Users\jing\AppData\Local\Temp\ipykernel_7616\432135112.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df2.groupby('link_ID').apply(mean_time)


In [145]:
import numpy as np
sorted_link = np.sort(df2['link_ID_en'].unique())

In [146]:
df2['link_ID_en'] = df2['link_ID_en'].map(lambda x: np.argmin(x >= sorted_link))

In [147]:
df2 = df2.reset_index(drop=True)
df2

,link_ID,date,travel_time,time_interval_begin,lagging1,lagging2,lagging3,lagging4,lagging5,length,area,vacation,minute_series,day_of_week,day_of_week_en,hour_en,"week_hour_1.0,1.0",links_num_2,width_3,link_ID_en
0,3377906280028510514,2017-03-01,1.662360,2017-03-01 06:00:00,NaN,NaN,NaN,NaN,NaN,48,144,0.0,0.0,3,1.0,1.0,True,True,True,0
1,3377906280028510514,2017-03-01,1.681661,2017-03-01 06:02:00,1.662360,NaN,NaN,NaN,NaN,48,144,0.0,2.0,3,1.0,1.0,True,True,True,0
2,3377906280028510514,2017-03-01,1.676155,2017-03-01 06:04:00,1.681661,1.662360,NaN,NaN,NaN,48,144,0.0,4.0,3,1.0,1.0,True,True,True,0
3,3377906280028510514,2017-03-01,1.683786,2017-03-01 06:06:00,1.676155,1.681661,1.662360,NaN,NaN,48,144,0.0,6.0,3,1.0,1.0,True,True,True,0
4,3377906280028510514,2017-03-01,1.683193,2017-03-01 06:08:00,1.683786,1.676155,1.681661,1.662360,NaN,48,144,0.0,8.0,3,1.0,1.0,True,True,True,0
5,3377906280028510514,2017-03-01,1.629241,2017-03-01 06:10:00,1.683193,1.683786,1.676155,1.681661,1.662360,48,144,0.0,10.0,3,1.0,1.0,True,True,True,0
6,3377906280028510514,2017-03-01,1.629241,2017-03-01 06:12:00,1.629241,1.683193,1.683786,1.676155,1.681661,48,144,0.0,12.0,3,1.0,1.0,True,True,True,0
7,3377906280028510514,2017-03-01,1.629241,2017-03-01 06:14:00,1.629241,1.629241,1.683193,1.683786,1.676155,48,144,0.0,14.0,3,1.0,1.0,True,True,True,0
8,3377906280028510514,2017-03-01,1.701797,2017-03-01 06:16:00,1.629241,1.629241,1.629241,1.683193,1.683786,48,144,0.0,16.0,3,1.0,1.0,True,True,True,0


In [148]:
df1_set = {'width_15', 'hour_en', 'week_hour_2.0,2.0', 'week_hour_3.0,3.0', 'area', 'width_9', 'week_hour_2.0,1.0', 'time_interval_begin', 'width_12', 'week_hour_3.0,1.0', 'length', 'date', 'links_num_2', 'week_hour_1.0,1.0', 'links_num_3', 'week_hour_2.0,3.0', 'lagging3', 'lagging4', 'link_ID', 'week_hour_1.0,2.0', 'links_num_4', 'week_hour_1.0,3.0', 'imputation1', 'width_6', 'links_num_5', 'day_of_week_en', 'minute_series', 'vacation', 'lagging2', 'width_3', 'week_hour_3.0,2.0', 'lagging1', 'travel_time', 'day_of_week', 'link_ID_en', 'lagging5'}

In [149]:
all_columns = set(df1_set).union(df2.columns)

# 扩展 df2，添加 df1 中的列，缺失的列填充为 0
df2 = df2.reindex(columns=all_columns, fill_value=0)

df2

,"week_hour_1.0,1.0",width_6,lagging2,"week_hour_3.0,1.0",travel_time,width_3,time_interval_begin,width_15,lagging4,"week_hour_3.0,2.0","week_hour_2.0,1.0",day_of_week_en,width_12,minute_series,area,lagging1,day_of_week,links_num_5,lagging3,lagging5,vacation,date,"week_hour_1.0,3.0",imputation1,links_num_2,"week_hour_3.0,3.0",link_ID,hour_en,"week_hour_2.0,2.0","week_hour_2.0,3.0",width_9,"week_hour_1.0,2.0",links_num_3,length,link_ID_en,links_num_4
0,True,0,NaN,0,1.662360,True,2017-03-01 06:00:00,0,NaN,0,0,1.0,0,0.0,144,NaN,3,0,NaN,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
1,True,0,NaN,0,1.681661,True,2017-03-01 06:02:00,0,NaN,0,0,1.0,0,2.0,144,1.662360,3,0,NaN,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
2,True,0,1.662360,0,1.676155,True,2017-03-01 06:04:00,0,NaN,0,0,1.0,0,4.0,144,1.681661,3,0,NaN,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
3,True,0,1.681661,0,1.683786,True,2017-03-01 06:06:00,0,NaN,0,0,1.0,0,6.0,144,1.676155,3,0,1.662360,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
4,True,0,1.676155,0,1.683193,True,2017-03-01 06:08:00,0,1.662360,0,0,1.0,0,8.0,144,1.683786,3,0,1.681661,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
5,True,0,1.683786,0,1.629241,True,2017-03-01 06:10:00,0,1.681661,0,0,1.0,0,10.0,144,1.683193,3,0,1.676155,1.662360,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
6,True,0,1.683193,0,1.629241,True,2017-03-01 06:12:00,0,1.676155,0,0,1.0,0,12.0,144,1.629241,3,0,1.683786,1.681661,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
7,True,0,1.629241,0,1.629241,True,2017-03-01 06:14:00,0,1.683786,0,0,1.0,0,14.0,144,1.629241,3,0,1.683193,1.676155,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
8,True,0,1.629241,0,1.701797,True,2017-03-01 06:16:00,0,1.683193,0,0,1.0,0,16.0,144,1.629241,3,0,1.629241,1.683786,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0


In [150]:

column_names = set(df1_set)

print(column_names)

column_names2 = set(df2.columns)

print(column_names2)

print(column_names-column_names2)
print(column_names2-column_names)
    

{'week_hour_1.0,1.0', 'width_6', 'lagging2', 'week_hour_3.0,1.0', 'travel_time', 'width_3', 'time_interval_begin', 'width_15', 'lagging4', 'week_hour_3.0,2.0', 'week_hour_2.0,1.0', 'day_of_week_en', 'width_12', 'minute_series', 'area', 'lagging1', 'day_of_week', 'links_num_5', 'lagging3', 'lagging5', 'vacation', 'date', 'week_hour_1.0,3.0', 'imputation1', 'links_num_2', 'week_hour_3.0,3.0', 'link_ID', 'hour_en', 'week_hour_2.0,2.0', 'week_hour_2.0,3.0', 'width_9', 'week_hour_1.0,2.0', 'links_num_3', 'length', 'link_ID_en', 'links_num_4'}
{'week_hour_1.0,1.0', 'width_6', 'lagging2', 'week_hour_3.0,1.0', 'travel_time', 'width_3', 'time_interval_begin', 'width_15', 'lagging4', 'week_hour_3.0,2.0', 'week_hour_2.0,1.0', 'day_of_week_en', 'width_12', 'minute_series', 'links_num_4', 'area', 'lagging1', 'day_of_week', 'links_num_5', 'lagging3', 'lagging5', 'vacation', 'date', 'week_hour_1.0,3.0', 'links_num_2', 'week_hour_3.0,3.0', 'link_ID', 'hour_en', 'week_hour_2.0,2.0', 'week_hour_2.0,3.0'

In [157]:
df = df2.copy()

lagging = 5
lagging_feature = ['lagging%01d' % e for e in range(lagging, 0, -1)]
lagging_feature

base_feature = [x for x in df.columns.values.tolist() if x not in ['time_interval_begin', 'link_ID', 'link_ID_int',
                                                                   'date', 'travel_time', 'imputation1',
                                                                   'minute_series', 'area', 'hour_en', 'day_of_week']]

base_feature = [x for x in base_feature if x not in lagging_feature]

train_feature = list(base_feature)
train_feature.extend(lagging_feature)

print(train_feature)

# test_df = df2.iloc[[-1]].copy()
test_df = df2.copy()
test_df

['week_hour_1.0,1.0', 'width_6', 'week_hour_3.0,1.0', 'width_3', 'width_15', 'week_hour_3.0,2.0', 'week_hour_2.0,1.0', 'day_of_week_en', 'width_12', 'links_num_5', 'vacation', 'week_hour_1.0,3.0', 'links_num_2', 'week_hour_3.0,3.0', 'week_hour_2.0,2.0', 'week_hour_2.0,3.0', 'width_9', 'week_hour_1.0,2.0', 'links_num_3', 'length', 'link_ID_en', 'links_num_4', 'lagging5', 'lagging4', 'lagging3', 'lagging2', 'lagging1']


,"week_hour_1.0,1.0",width_6,lagging2,"week_hour_3.0,1.0",travel_time,width_3,time_interval_begin,width_15,lagging4,"week_hour_3.0,2.0","week_hour_2.0,1.0",day_of_week_en,width_12,minute_series,area,lagging1,day_of_week,links_num_5,lagging3,lagging5,vacation,date,"week_hour_1.0,3.0",imputation1,links_num_2,"week_hour_3.0,3.0",link_ID,hour_en,"week_hour_2.0,2.0","week_hour_2.0,3.0",width_9,"week_hour_1.0,2.0",links_num_3,length,link_ID_en,links_num_4
0,True,0,NaN,0,1.662360,True,2017-03-01 06:00:00,0,NaN,0,0,1.0,0,0.0,144,NaN,3,0,NaN,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
1,True,0,NaN,0,1.681661,True,2017-03-01 06:02:00,0,NaN,0,0,1.0,0,2.0,144,1.662360,3,0,NaN,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
2,True,0,1.662360,0,1.676155,True,2017-03-01 06:04:00,0,NaN,0,0,1.0,0,4.0,144,1.681661,3,0,NaN,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
3,True,0,1.681661,0,1.683786,True,2017-03-01 06:06:00,0,NaN,0,0,1.0,0,6.0,144,1.676155,3,0,1.662360,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
4,True,0,1.676155,0,1.683193,True,2017-03-01 06:08:00,0,1.662360,0,0,1.0,0,8.0,144,1.683786,3,0,1.681661,NaN,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
5,True,0,1.683786,0,1.629241,True,2017-03-01 06:10:00,0,1.681661,0,0,1.0,0,10.0,144,1.683193,3,0,1.676155,1.662360,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
6,True,0,1.683193,0,1.629241,True,2017-03-01 06:12:00,0,1.676155,0,0,1.0,0,12.0,144,1.629241,3,0,1.683786,1.681661,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
7,True,0,1.629241,0,1.629241,True,2017-03-01 06:14:00,0,1.683786,0,0,1.0,0,14.0,144,1.629241,3,0,1.683193,1.676155,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0
8,True,0,1.629241,0,1.701797,True,2017-03-01 06:16:00,0,1.683193,0,0,1.0,0,16.0,144,1.629241,3,0,1.629241,1.683786,0.0,2017-03-01,0,0,True,0,3377906280028510514,1.0,0,0,0,0,0,48,0,0


In [158]:
import joblib

test_X = test_df[train_feature]

regressor = joblib.load('model/xgbr.pkl')
y_prediction = regressor.predict(test_X.values)

test_df['lagging5'] = test_df['lagging4']
test_df['lagging4'] = test_df['lagging3']
test_df['lagging3'] = test_df['lagging2']
test_df['lagging2'] = test_df['lagging1']
test_df['lagging1'] = y_prediction

test_df['predicted'] = np.expm1(y_prediction)
test_df['time_interval_begin'] = test_df['time_interval_begin'] + pd.DateOffset(minutes=2)
test_df['time_interval'] = test_df['time_interval_begin'].map(
    lambda x: '[' + str(x) + ',' + str(x + pd.DateOffset(minutes=2)) + ')')
test_df.time_interval = test_df.time_interval.astype(object)

test_df[['link_ID', 'date', 'time_interval', 'predicted']]

,link_ID,date,time_interval,predicted
0,3377906280028510514,2017-03-01,"[2017-03-01 06:02:00,2017-03-01 06:04:00)",146.875519
1,3377906280028510514,2017-03-01,"[2017-03-01 06:04:00,2017-03-01 06:06:00)",14.344766
2,3377906280028510514,2017-03-01,"[2017-03-01 06:06:00,2017-03-01 06:08:00)",5.309178
3,3377906280028510514,2017-03-01,"[2017-03-01 06:08:00,2017-03-01 06:10:00)",4.549357
4,3377906280028510514,2017-03-01,"[2017-03-01 06:10:00,2017-03-01 06:12:00)",4.496519
5,3377906280028510514,2017-03-01,"[2017-03-01 06:12:00,2017-03-01 06:14:00)",3.056073
6,3377906280028510514,2017-03-01,"[2017-03-01 06:14:00,2017-03-01 06:16:00)",2.946508
7,3377906280028510514,2017-03-01,"[2017-03-01 06:16:00,2017-03-01 06:18:00)",2.933948
8,3377906280028510514,2017-03-01,"[2017-03-01 06:18:00,2017-03-01 06:20:00)",2.944465
